In [ ]:
#anqi wu
#12.08.2024

#idea on tf-idf
#idea on fine-tuning bert on 900 labeled dataset

In [ ]:
#file preparation
#TF-IDF computation file only with labeled dataset
#this can be find in merged-data

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# load merged data
merged_data_path = '/content/merged_data.csv'  # please locate this in our shared folder
merged_data = pd.read_csv(merged_data_path)

label_column = 'label_x'
drop_columns = ['url', 'label_x']

# #debug
# if labeled_data.empty:
#     raise ValueError("No labeled data available for training.")

#training features
features = labeled_data.drop(columns=drop_columns).select_dtypes(include=['float64', 'int64']).fillna(0)
labels = labeled_data[label_column]

#split and train
X_train, X_val, y_train, y_val = train_test_split(features, labels, test_size=0.2, random_state=42)

#random forest
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)
#validation
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))


In [ ]:
#logisticRegression
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
#gradient boosting
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
#SVM
from sklearn.svm import SVC

model = SVC(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
#kmean
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
#cnn
from sklearn.neural_network import MLPClassifier

model = MLPClassifier(random_state=42, max_iter=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
print(classification_report(y_val, y_pred))

In [ ]:
###########################################
#pretained on bert
#gpu
#make sure installation
#pip install transformers datasets torch pandas scikit-learn

import pandas as pd
# load
file_path = 'label_tokenized.xlsx' #refer this in our shared folder #file name should be this
data = pd.read_excel(file_path)
print(data.head())

In [ ]:
#only put high accuracy one
#please refer notebook for other pretained model
from sklearn.utils import resample

#oversampling minority approach
majority_class = data[data['label'] == 2]  # Negative
minority_class_1 = data[data['label'] == 0]  # No Attitude
minority_class_2 = data[data['label'] == 1]  # Positive
oversampled_class_1 = resample(minority_class_1, replace=True, n_samples=len(majority_class), random_state=42)
oversampled_class_2 = resample(minority_class_2, replace=True, n_samples=len(majority_class), random_state=42)

balanced_data = pd.concat([majority_class, oversampled_class_1, oversampled_class_2])
balanced_data = balanced_data.sample(frac=1, random_state=42).reset_index(drop=True)
print(balanced_data['label'].value_counts())

In [ ]:
data = balanced_data

#refer from tutorial and hw4
from transformers import BertTokenizer
from datasets import Dataset

# initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# create dataset dictionary from oversampled data
data_dict = {
    "text": balanced_data['processed_body'].tolist(),
    "label": balanced_data['label'].tolist()
}

# convert to Hugging Face
dataset = Dataset.from_dict(data_dict)

# tokenization function
def tokenize_function(batch):
    return tokenizer(batch['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# split into train and test datasets
train_test_split = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=5,  #epoch was increased from 3 to 5, could be overfitting
    weight_decay=0.01,
    save_steps=500,
    save_total_limit=2,
    logging_dir='./logs',
)

In [ ]:
from transformers import BertForSequenceClassification, Trainer
import torch

# initialization
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

#class weighted differently
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(balanced_data['label']),
    y=balanced_data['label']
)

# class weights to tensor
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)

#customer weighted labels
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights_tensor.to(model.device))
        loss = loss_fn(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)
trainer.train()


In [ ]:
from sklearn.metrics import classification_report

# prediction
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(axis=-1)
true_labels = predictions.label_ids

#validation report
print(classification_report(true_labels, predicted_labels, target_names=["No Attitude", "Positive", "Negative"]))
